In [3]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

# Streamlit 제목
st.title("Heart Disease Prediction and Analysis")

# 글자 처리
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
pd.options.display.float_format = '{:.2f}'.format

# 데이터 로드
st.subheader("1. 데이터 로드 및 샘플")
data = pd.read_csv('./dataset/heart_disease.csv')
st.write("데이터 샘플:")
st.dataframe(data.head())

# 결측값 처리
st.subheader("2. 결측값 처리")
if data.isnull().sum().sum() > 0:
    numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
    categorical_columns = data.select_dtypes(include=['object']).columns

    data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())
    data[categorical_columns] = data[categorical_columns].fillna(data[categorical_columns].mode().iloc[0])

st.write("결측값 처리 완료. 총 결측값 수:", data.isnull().sum().sum())

# 데이터 인코딩
st.subheader("3. 범주형 데이터 인코딩")
label_encoder = LabelEncoder()
categorical_columns = data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])

st.write("인코딩 완료:")
st.dataframe(data.head())

# 독립변수(X)와 종속변수(y) 분리
st.subheader("4. 데이터 분리")
X = data.drop(columns=['Heart Disease Status'])
y = data['Heart Disease Status']

# 데이터 불균형 해결 및 분리
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
st.write("SMOTE를 사용한 데이터 분리 완료:")
st.write(f"훈련 데이터 크기: {X_train.shape}")
st.write(f"테스트 데이터 크기: {X_test.shape}")

# 랜덤 포레스트 모델 학습
st.subheader("5. 모델 학습 및 평가")
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
st.write(f"모델 정확도: {accuracy * 100:.2f}%")

# 교차 검증
scores = cross_val_score(model, X_resampled, y_resampled, cv=5, scoring='accuracy')
st.write(f"Cross-Validation 평균 정확도: {scores.mean() * 100:.2f}%")

# 분류 보고서
st.write("분류 보고서:")
st.text(classification_report(y_test, y_pred, target_names=['질병 없음', '질병 있음']))

# F1-Score 요약
report = classification_report(y_test, y_pred, target_names=['질병 없음', '질병 있음'], output_dict=True)
f1_score_disease = report['질병 있음']['f1-score']
st.write(f"질병 있음 F1-Score: {f1_score_disease:.2f}")

# 타겟 변수 분포 시각화 (나이를 x축으로)
st.subheader("6. 타겟 변수 분포")
fig1, ax1 = plt.subplots(figsize=(8, 6))
sns.countplot(data['Heart Disease Status'], ax=ax1)
ax1.set_title("Distribution of Heart Disease Status")
ax1.set_xlabel("Heart Disease (0: No, 1: Yes)")
ax1.set_ylabel("Count")
st.pyplot(fig1)


# 상관관계 히트맵
st.subheader("7. 상관관계 히트맵")
numeric_data = data.select_dtypes(include=['number'])
correlation_matrix = numeric_data.corr()
fig2, ax2 = plt.subplots(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', square=True, ax=ax2)
ax2.set_title("변수별 상관관계 히트맵")
st.pyplot(fig2)

# 주요 변수와 타겟 간 관계
st.subheader("8. 주요 변수와 타겟 변수 간 관계")
fig3, ax3 = plt.subplots(figsize=(8, 6))
sns.boxplot(x='Heart Disease Status', y='Cholesterol Level', data=data, ax=ax3)
ax3.set_title("Cholesterol Level by Heart Disease Status")
ax3.set_xlabel("Heart Disease Status")
ax3.set_ylabel("Cholesterol Level")
st.pyplot(fig3)

# 혼동 행렬 시각화
st.subheader("9. 혼동 행렬")
cm = confusion_matrix(y_test, y_pred)
fig4, ax4 = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Disease', 'Disease'], yticklabels=['No Disease', 'Disease'], ax=ax4)
ax4.set_title("혼동 행렬")
ax4.set_xlabel("예측값")
ax4.set_ylabel("실제값")
st.pyplot(fig4)

# 특성 중요도 시각화
st.subheader("10. 특성 중요도")
feature_importances = model.feature_importances_
features = X.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances}).sort_values(by='Importance', ascending=False)
top_features = importance_df.head(10)

fig5, ax5 = plt.subplots(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=top_features, ax=ax5)
ax5.set_title("Feature Importances")
ax5.set_xlabel("Importance")
ax5.set_ylabel("Features")
st.pyplot(fig5)

# 실제값과 예측값 비교 시각화
st.subheader("11. 실제값과 예측값 비교")
fig6, ax6 = plt.subplots(figsize=(8, 6))
sns.histplot(y_test, label='실제값', color='blue', kde=False, bins=2, alpha=0.6)
sns.histplot(y_pred, label='예측값', color='orange', kde=False, bins=2, alpha=0.6)
ax6.set_title("Actual vs Predicted Distribution")
ax6.legend()
st.pyplot(fig6)


2025-01-06 14:10:48.217 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-06 14:10:48.218 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-06 14:10:48.219 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-06 14:10:48.219 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-06 14:10:48.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-06 14:10:48.261 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-06 14:10:48.262 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-06 14:10:48.263 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()